In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Prerequisites

For this tutorial I reccomend you create new conda anvironment
```bash
conda create -n edu python=3.9 jupyter matplotlib
```

```bash
conda activate edu
```

## Install OpenMM

```bash 
! pip install autopep8
```

```bash
! pip install opencv-python
```

```bash
! pip install -U openmim
! mim install mmcv-full
```

## Install MMSegmentation

```bash
! git clone https://github.com/open-mmlab/mmsegmentation.git
! cd mmsegmentation
! pip install -v -e .
# "-v" means verbose, or more output
# "-e" means installing a project in editable mode,
# thus any local modifications made to the code will take effect without reinstallation.
```

```bash
! pip install mmsegmentation
```

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

# Semantic segmentation

## Organize import

In [ ]:
from pathlib import Path

In [ ]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette

In [ ]:
from mmseg.apis import inference_segmentor, init_segmentor
import mmcv

In [ ]:
from multiprocessing import cpu_count

In [ ]:
import torch

In [ ]:
## Configure workers and batch size

In [ ]:
workers = cpu_count()
workers

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
bs = 32

#### Verify installation

In [ ]:
segpath = Path('../mmsegmentation')

```bash
! mim download mmsegmentation --config pspnet_r50-d8_512x1024_40k_cityscapes --dest {segpath}
```

In [ ]:
! ls {segpath}

In [ ]:
config_file = segpath / 'pspnet_r50-d8_512x1024_40k_cityscapes.py'
checkpoint_file = segpath / 'pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth'

In [ ]:
# build the model from a config file and a checkpoint file
model = init_segmentor(str(config_file), str(checkpoint_file), device=device)

# test a single image and show the results
img = str(segpath / 'demo' / 'demo.png') #'test.jpg'  # or img = mmcv.imread(img), which will only load it once
result = inference_segmentor(model, img)
show_result_pyplot(model, img, result, get_palette('cityscapes'))

## Download data

In [ ]:
path = Path('data')
camvid = path / 'camvid'
camvid.mkdir(exist_ok = True)

In [ ]:
! ls {path}

In [ ]:
if not (camvid / 'camvid_tiny.tgz').exists():
    ! wget http://files.fast.ai/data/examples/camvid_tiny.tgz -P {camvid}

In [ ]:
! ls {camvid}

In [ ]:
camvid_tiny = camvid / 'camvid_tiny'

In [ ]:
if not camvid_tiny.exists():
    ! tar xf {camvid / 'camvid_tiny.tgz'} -C {camvid}

In [ ]:
! ls {camvid_tiny}

In [ ]:
! ls {camvid_tiny / 'images'}

In [ ]:
# Let's take a look at the dataset
import mmcv
import matplotlib.pyplot as plt

img = mmcv.imread(camvid_tiny / 'images/' / '0016E5_07290.png')
plt.figure(figsize=(8, 6))
plt.imshow(mmcv.bgr2rgb(img))
plt.show()

In [ ]:
palette_path = camvid_tiny / 'label_colors.txt'

In [ ]:
if not palette_path.exists():
    ! wget http://mi.eng.cam.ac.uk/research/projects/VideoRec/CamVid/data/label_colors.txt -P {camvid_tiny}

In [ ]:
import re

In [ ]:
with palette_path.open('r') as plts:
    lines = plts.readlines()
    classes = list()
    palette = list()
    for idx, line in enumerate(lines):
        cl_ln = line.strip()
        pltte = re.findall('[0-9]+', cl_ln)
        cl_nm = re.findall('[A-Z-a-z]+', cl_ln)[0]
        palette.append(pltte)
        classes.append(cl_nm)
        print(idx + 1, pltte, cl_nm)

In [ ]:
! ls {camvid_tiny / 'labels'}

#### Convert dataset

In [ ]:
import os.path as osp

In [ ]:
data_root = camvid_tiny
img_dir = 'images'
ann_dir = 'labels'

In [ ]:
# split train/val set randomly
split_dir = 'splits'
mmcv.mkdir_or_exist(data_root / split_dir)
filename_list = [osp.splitext(filename)[0] for filename in mmcv.scandir(
    osp.join(data_root, img_dir), suffix='.png')]
with open(osp.join(data_root, split_dir, 'train.txt'), 'w') as f:
  # select first 4/5 as train set
  train_length = int(len(filename_list)*4/5)
  f.writelines(line + '\n' for line in filename_list[:train_length])
with open(osp.join(data_root, split_dir, 'val.txt'), 'w') as f:
  # select last 1/5 as train set
  f.writelines(line + '\n' for line in filename_list[train_length:])

In [ ]:
! ls {data_root / split_dir}

In [ ]:
with (data_root / split_dir / 'train.txt').open('r') as spl:
    lines = spl.readlines()
    for line in lines:
        print(line)

In [ ]:
with (data_root / split_dir / 'val.txt').open('r') as spl:
    lines = spl.readlines()
    for line in lines:
        print(line)

#### Registar dataset

In [ ]:
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset

@DATASETS.register_module(force=True)
class CamvidDataset(CustomDataset):
  CLASSES = classes
  PALETTE = palette
  def __init__(self, split, **kwargs):
    super().__init__(img_suffix='.png', seg_map_suffix='_P.png', 
                     split=split, **kwargs)
    assert osp.exists(self.img_dir) and self.split is not None


#### Visualize dataset

In [ ]:
import matplotlib.patches as mpatches
from PIL import Image
import numpy as np

In [ ]:
# # Let's take a look at the segmentation map we got
# img = Image.open(data_root/ ann_dir / '0016E5_07290_P.png')
# plt.figure(figsize=(8, 6))
# im = plt.imshow(np.array(img.convert('RGB')))

# # create a patch (proxy artist) for every color 
# patches = [mpatches.Patch(color=np.array(palette[i])/255., 
#                           label=classes[i]) for i in range(32)]
# # put those patched as legend-handles into the legend
# plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., 
#            fontsize='large')

# plt.show()

## Download and configure checkpoint

In [ ]:
checkpoints = Path('checkpoints')
checkpoints.mkdir(exist_ok=True)

In [ ]:
psp_city_ckp = 'pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth'
psp_city_path = checkpoints / psp_city_ckp

In [ ]:
if not psp_city_path.exists():
    !wget https://download.openmmlab.com/mmsegmentation/v0.5/pspnet/pspnet_r50-d8_512x1024_40k_cityscapes/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth -P checkpoints

In [ ]:
! ls {checkpoints}

## Prepare configuration

In [ ]:
from mmcv import Config
from mmseg.apis import set_random_seed

In [ ]:
conf_path = segpath / 'configs'

In [ ]:
! ls {conf_path}

In [ ]:
psp_config = conf_path / 'pspnet'
psp_config_city = psp_config / 'pspnet_r50-d8_512x1024_40k_cityscapes.py'

In [ ]:
cfg = Config.fromfile(str(psp_config_city))

In [ ]:
print(cfg.pretty_text)

In [ ]:
! ls {checkpoints}

In [ ]:
# Since we use only one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='BN', requires_grad=True)
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg
# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 8
cfg.model.auxiliary_head.num_classes = 8
cfg.model.init_cfg=dict(
    type='Pretrained', 
    checkpoint='open-mmlab://resnet50_v1c'),

# Modify dataset type and path
cfg.dataset_type = 'CamvidDataset'
cfg.data_root = str(data_root)

cfg.data.samples_per_gpu = 32
cfg.data.workers_per_gpu=8

cfg.img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
cfg.crop_size = (256, 256)
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(320, 240), ratio_range=(0.5, 2.0)),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(type='Normalize', **cfg.img_norm_cfg),
    dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(320, 240),
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]


cfg.data.train.type = cfg.dataset_type
cfg.data.train.data_root = cfg.data_root
cfg.data.train.img_dir = img_dir
cfg.data.train.ann_dir = ann_dir
cfg.data.train.pipeline = cfg.train_pipeline
cfg.data.train.split = 'splits/train.txt'

cfg.data.val.type = cfg.dataset_type
cfg.data.val.data_root = cfg.data_root
cfg.data.val.img_dir = img_dir
cfg.data.val.ann_dir = ann_dir
cfg.data.val.pipeline = cfg.test_pipeline
cfg.data.val.split = 'splits/val.txt'

cfg.data.test.type = cfg.dataset_type
cfg.data.test.data_root = cfg.data_root
cfg.data.test.img_dir = img_dir
cfg.data.test.ann_dir = ann_dir
cfg.data.test.pipeline = cfg.test_pipeline
cfg.data.test.split = 'splits/val.txt'

# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = str(checkpoints / 
                    'pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth')

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/tutorial'

cfg.runner.max_iters = 256
cfg.log_config.interval = 32
cfg.evaluation.interval = 128
cfg.checkpoint_config.interval = 256

# Set seed to facitate reproducing the result
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device = device
cfg.cudnn_benchmark = True

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

In [ ]:
try:
    del cfg.model.backbone.pretrained
except:
    print('cfg.model.backbone.pretrained noes not exists')

In [ ]:
try:
    del cfg.model.pretrained
except:
    print('cfg.model.pretrained noes not exists')

In [ ]:
cfg.model.backbone

``` bash
! cp {segpath / 'demo' / 'MMSegmentation_Tutorial.ipynb'} 'mmsegmentation_tutorial.ipynb'
```

## Train model

In [ ]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor


# Build the dataset
datasets = [build_dataset(cfg.data.train)]

#### Build model

In [ ]:
# Build the detector
model = build_segmentor(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

#### Run training

In [ ]:
# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=True, 
                meta=dict())

## Inference on trained model

In [ ]:
model

In [ ]:
img = mmcv.imread(str(data_root / img_dir / 'Seq05VD_f01080.png'))

In [ ]:
img.shape

In [ ]:
model.cfg = cfg

In [ ]:
result = inference_segmentor(model, img)
plt.figure(figsize=(8, 6))
show_result_pyplot(model, img, result, palette)

# Object detection with MMDetection library

The library MMDetection in part of the OpenMMLab library

#### Install library

```python
pip install -U openmim
mim install mmcv-full
```

In [ ]:
! ls ~/git

```python
! git clone https://github.com/open-mmlab/mmdetection.git ../mmdetection
```

``` python
! cd ../mmdetection; pip install -U -v -e .
```

## Organize imports

In [ ]:
import mmcv
from mmcv.runner import load_checkpoint

from mmdet.apis import train_detector, inference_detector, show_result_pyplot, set_random_seed
from mmdet.models import build_detector

In [ ]:
from mmcv import Config

In [ ]:
import mmcv
import numpy as np

from mmdet.datasets import build_dataset
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

In [ ]:
import torch

In [ ]:
import pandas as pd

In [ ]:
import json
from tqdm import tqdm
import pickle
import zipfile
import os

In [ ]:
from pathlib import Path

In [ ]:
from multiprocessing import cpu_count

In [ ]:
## Configure workers and batch size

In [ ]:
workers = cpu_count()
workers

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
bs = 32

#### Verify installation

In [ ]:
detpath = Path('../mmdetection')

In [ ]:
det_config = detpath / 'configs'

In [ ]:
frcnn_config = det_config / 'faster_rcnn' / 'faster_rcnn_r50_caffe_fpn_mstrain_1x_coco.py'

In [ ]:
fcnn_checkpoints = Path('checkpoints')
fcnn_checkpoints.mkdir(exist_ok=True)

In [ ]:
! ls {fcnn_checkpoints}

In [ ]:
fcnn_checkpoints_path = fcnn_checkpoints / 'faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth'

In [ ]:
if not fcnn_checkpoints_path.exists():
    !wget -c https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth \
      -O {fcnn_checkpoints_path}

In [ ]:
! ls {det_config}

In [ ]:
! ls {frcnn_config}

In [ ]:
# Load the config
config = mmcv.Config.fromfile(frcnn_config)
# Set pretrained to be None since we do not need pretrained model here
config.model.pretrained = None

# Initialize the detector
model = build_detector(config.model)

# Load checkpoint
checkpoint = load_checkpoint(model, str(fcnn_checkpoints_path), map_location=device)

# Set the classes of models for inference
model.CLASSES = checkpoint['meta']['CLASSES']

# We need to set the model's cfg for inference
model.cfg = config

# Convert the model to GPU
model.to(device)
# Convert the model into evaluation mode
model.eval()

In [ ]:
# Use the detector to do inference
img = detpath / 'demo' / 'demo.jpg'
result = inference_detector(model, str(img))

In [ ]:
# Let's plot the result
show_result_pyplot(model, img, result, score_thr=0.3)

#### Prepare dataset

In [ ]:
! pip install -U kaggle

In [ ]:
Path.home()

In [ ]:
path = Path.home() /'.kaggle'
path.mkdir(exist_ok=True)

In [ ]:
datasets = Path('data') / 'wheat'
datasets.mkdir(exist_ok=True)

In [ ]:
! ls {datasets}

In [ ]:
wheet_path = datasets / 'global-wheat-detection.zip'
wheet_path

In [ ]:
zip_not_exists = not wheet_path.exists()

In [ ]:
json_path = path / 'kaggle.json'
if zip_not_exists:
    api_token = {"username":"yout-user","key":"your-key"}
    with json_path.open(mode='w') as file:
        json.dump(api_token, file)

In [ ]:
json_path

In [ ]:
if zip_not_exists:
    ! chmod 600 ~/.kaggle/kaggle.json
    ! kaggle datasets list

In [ ]:
if zip_not_exists:
    ! kaggle competitions download -c global-wheat-detection -p {datasets}

```python
datasets = Path('/content') / 'drive' / 'My\ Drive' / 'datasets'
! ls {datasets}
```

In [ ]:
data_path = datasets

In [ ]:
! ls {data_path}

In [ ]:
csv_file_not_exists = not (data_path / 'sample_submission.csv').exists()

In [ ]:
if csv_file_not_exists:
    zip_ref = zipfile.ZipFile(data_path / 'global-wheat-detection.zip', 'r')
    zip_ref.extractall(path=data_path)
    zip_ref.close()

In [ ]:
! ls {data_path}

#### Convert dataset

In [ ]:
images_path = data_path / 'train' 
csv_path = data_path / 'train.csv'

In [ ]:
df = pd.read_csv(csv_path)

In [ ]:
df

In [ ]:
class_names = df.source.unique().tolist()
classes = {class_name: idx for idx, class_name in enumerate(class_names)}
LABEL_NAMES = classes
LABEL_NAMES

In [ ]:
df.image_id.unique().shape

In [ ]:
def create_dataset(df):
    data_infos = []
    image_df = df.image_id.unique()
    items = len(image_df)
    image_df = enumerate(image_df)
    with tqdm(image_df, total=items, desc='converting data') as prog:
        for image_id, img_name in prog:
            record = dict()
            image_df = df[df.image_id == img_name]
            record['filename'] = f'{img_name}.jpg'
            record['image_id'] = image_id
            record['height'] = int(image_df.iloc[0].height)
            record['width'] = int(image_df.iloc[0].width)
            bboxes = list()
            labels = list()
            for _, row in image_df.iterrows():
                bbox_raw = json.loads(row.bbox)
                bbox = [int(bbox_raw[0]), int(bbox_raw[1]),
                        int(bbox_raw[0] + bbox_raw[2]), int(bbox_raw[1] + bbox_raw[3])]
                bboxes.append(bbox)
                category_id=classes.get(row.source, 0)
                labels.append(category_id)
            record['ann'] = dict(
                bboxes=np.array(bboxes).astype(np.float32),
                labels=np.array(labels).astype(np.int64))
            data_infos.append(record)

    return data_infos

In [ ]:
data_pickle = data_path / 'dataset.pkl'
train_pickle = data_path / 'train.pkl'
val_pickle = data_path / 'val.pkl'

In [ ]:
dataset_not_exists = not data_pickle.exists()
train__not_exists = not train_pickle.exists()
val_not_exists = not val_pickle.exists()

In [ ]:
if dataset_not_exists:
    data_dicts = create_dataset(df)
elif train__not_exists or val_not_exists:
    with data_pickle.open(mode='rb') as fp:
        data_dicts = pickle.load(fp)
else:
    print(f'All filrs: {data_pickle} {train_pickle} {val_pickle} already there')

In [ ]:
if train__not_exists or val_not_exists:
    data_dicts[-1]['filename']

In [ ]:
if train__not_exists or val_not_exists:
    total_data = len(data_dicts)
    split_data = total_data - (total_data// 4)

In [ ]:
if train__not_exists or val_not_exists:
    train_dicts = data_dicts[:split_data]
    val_dicts = data_dicts[split_data:]
    len(train_dicts), len(val_dicts)

In [ ]:
if dataset_not_exists:
    with data_pickle.open(mode='wb') as fp:
        pickle.dump(data_dicts, fp)

In [ ]:
if train__not_exists:
    with train_pickle.open(mode='wb') as fp:
        pickle.dump(train_dicts, fp)

In [ ]:
if val_not_exists:
    with val_pickle.open(mode='wb') as fp:
        pickle.dump(val_dicts, fp)

In [ ]:
with data_pickle.open(mode='rb') as fp:
    dataset = pickle.load(fp)

In [ ]:
with train_pickle.open(mode='rb') as fp:
    train_data = pickle.load(fp)

In [ ]:
with val_pickle.open(mode='rb') as fp:
    val_data = pickle.load(fp)

In [ ]:
len(train_data), len(val_data)

In [ ]:
@DATASETS.register_module(force=True)
class WheatDataset(CustomDataset):

    CLASSES = list(LABEL_NAMES.keys())

    def load_annotations(self, ann_file):
        with Path(ann_file).open(mode='rb') as fp:
            data_infos = pickle.load(fp)
        
        return data_infos

    def get_ann_info(self, idx):
        return self.data_infos[idx]['ann']

## Configure model

In [ ]:
cfg = Config.fromfile(str(frcnn_config))

In [ ]:
data_root = str(data_path)
data_root

In [ ]:
# Modify dataset type and path
cfg.dataset_type = 'WheatDataset'
cfg.data_root = data_root

cfg.data.test.type = 'WheatDataset'
cfg.data.test.data_root = data_root
cfg.data.test.ann_file = 'val.pkl'
cfg.data.test.img_prefix = 'train'

cfg.data.train.type = 'WheatDataset'
cfg.data.train.data_root = data_root
cfg.data.train.ann_file = 'train.pkl'
cfg.data.train.img_prefix = 'train'

cfg.data.val.type = 'WheatDataset'
cfg.data.val.data_root = data_root
cfg.data.val.ann_file = 'val.pkl'
cfg.data.val.img_prefix = 'train'

cfg.data.samples_per_gpu=bs // 4
cfg.data.workers_per_gpu=workers

# modify num classes of the model in box head
cfg.model.roi_head.bbox_head.num_classes = len(LABEL_NAMES)
# If we need to finetune a model based on a pre-trained detector, we need to
# use load_from to set the path of checkpoints.
cfg.load_from = str(fcnn_checkpoints_path)

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 32

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 6
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 12

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.cudnn_benchmark = True
cfg.device = device

# We can also use tensorboard to log the training process
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='TensorboardLoggerHook')]


# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

## Train the model

In [ ]:
import os.path as osp

In [ ]:
# Build dataset
datasets = [build_dataset(cfg.data.train)]

In [ ]:
# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

In [ ]:
# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

2022-06-22 14:54:59,497 - mmdet - INFO - Epoch [10][160/317]	lr: 2.500e-04, eta: 0:06:39, time: 0.540, data_time: 0.026, memory: 8226, loss_rpn_cls: 0.0378, loss_rpn_bbox: 0.1197, loss_cls: 0.2700, acc: 88.0592, loss_bbox: 0.4652, loss: 0.8927
2022-06-22 14:55:17,098 - mmdet - INFO - Epoch [10][192/317]	lr: 2.500e-04, eta: 0:06:23, time: 0.549, data_time: 0.026, memory: 8226, loss_rpn_cls: 0.0422, loss_rpn_bbox: 0.1199, loss_cls: 0.2765, acc: 87.8784, loss_bbox: 0.4710, loss: 0.9097
2022-06-22 14:55:34,654 - mmdet - INFO - Epoch [10][224/317]	lr: 2.500e-04, eta: 0:06:07, time: 0.549, data_time: 0.027, memory: 8226, loss_rpn_cls: 0.0404, loss_rpn_bbox: 0.1192, loss_cls: 0.2743, acc: 87.8891, loss_bbox: 0.4681, loss: 0.9020
2022-06-22 14:55:52,031 - mmdet - INFO - Epoch [10][256/317]	lr: 2.500e-04, eta: 0:05:51, time: 0.543, data_time: 0.025, memory: 8226, loss_rpn_cls: 0.0410, loss_rpn_bbox: 0.1182, loss_cls: 0.2744, acc: 88.0348, loss_bbox: 0.4608, loss: 0.8944
2022-06-22 14:56:09,198 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 843/843, 26.3 task/s, elapsed: 32s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-06-22 15:02:50,172 - mmdet - INFO - 
+-----------+-------+-------+--------+-------+
| class     | gts   | dets  | recall | ap    |
+-----------+-------+-------+--------+-------+
| usask_1   | 0     | 10133 | 0.000  | 0.000 |
| arvalis_1 | 0     | 536   | 0.000  | 0.000 |
| inrae_1   | 0     | 2199  | 0.000  | 0.000 |
| ethz_1    | 0     | 210   | 0.000  | 0.000 |
| arvalis_3 | 5538  | 32245 | 0.925  | 0.456 |
| rres_1    | 20236 | 0     | 0.000  | 0.000 |
| arvalis_2 | 4179  | 0     | 0.000  | 0.000 |
+-----------+-------+-------+--------+-------+
| mAP       |       |       |        | 0.152 |
+-----------+-------+-------+--------+-------+
2022-06-22 15:02:50,219 - mmdet - INFO - Epoch(val) [12][843]	AP50: 0.1520, mAP: 0.1521


#### Look into the training process

In [ ]:
# load tensorboard in colab
%load_ext tensorboard

# see curves in tensorboard
%tensorboard --logdir ./tutorial_exps --port 8899

## Inference with model

In [ ]:
test_path = data_path / 'test'

In [ ]:
! ls {test_path}

In [ ]:
img = mmcv.imread(str(test_path /'2fd875eaa.jpg'))

In [ ]:
model.cfg = cfg

In [ ]:
result = inference_detector(model, img)
show_result_pyplot(model, img, result)

## Questions?